In [ ]:
"""
================================================================================
QUANTUM COMPUTING PROJECT: Historical Data Analysis with PennyLane
================================================================================

This notebook demonstrates the power of quantum computing through:
1. Quantum Machine Learning (QML): Predicting gladiator survival using Variational Quantum Classifiers
2. Quantum Optimization: Finding optimal groupings in historical Wikipedia data using QAOA

Author: Quantum Historical Data Search Project
================================================================================
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# PennyLane imports for quantum computing
import pennylane as qml
from pennylane import numpy as pnp

# Set random seeds for reproducibility
np.random.seed(42)
pnp.random.seed(42)

print("=" * 80)
print("QUANTUM COMPUTING PROJECT: Historical Data Analysis")
print("=" * 80)
print("\nLibraries imported successfully!")
print(f"PennyLane version: {qml.__version__}")

# Part 1: Data Loading and Exploration

Let's start by loading and exploring our historical datasets.


In [ ]:
# Load the gladiator dataset
# This dataset contains information about ancient Roman gladiators
gladiator_df = pd.read_csv("gladiator_data.csv")

print("=" * 80)
print("GLADIATOR DATASET OVERVIEW")
print("=" * 80)
print(f"Shape: {gladiator_df.shape}")
print(f"\nColumns: {list(gladiator_df.columns)}")
print(f"\nFirst few rows:")
print(gladiator_df.head())
print(f"\nSurvival rate: {gladiator_df['Survived'].mean():.2%}")
print(f"\nMissing values:\n{gladiator_df.isnull().sum().sum()}")


In [ ]:
# Load the Wikipedia historical dataset
wiki_df = pd.read_csv("wiki_data.csv")

print("=" * 80)
print("WIKIPEDIA HISTORICAL DATASET OVERVIEW")
print("=" * 80)
print(f"Shape: {wiki_df.shape}")
print(f"\nColumns: {list(wiki_df.columns)}")
print(f"\nFirst few entries:")
print(wiki_df[['title', 'relevans', 'popularity', 'ranking']].head(10))
print(f"\nMissing values:\n{wiki_df.isnull().sum()}")


# Part 2: Quantum Machine Learning (QML) - Gladiator Survival Prediction

## Overview
We'll use a **Variational Quantum Classifier (VQC)** to predict whether a gladiator survived based on their characteristics. This demonstrates how quantum circuits can learn patterns in classical data.

### Key Concepts:
- **Variational Quantum Circuits (VQC)**: Parameterized quantum circuits that can be trained like neural networks
- **Feature Encoding**: Mapping classical data to quantum states using angle encoding
- **Variational Layers**: Parameterized rotations that learn optimal transformations
- **Measurement**: Extracting classical predictions from quantum states


In [ ]:
# ============================================================================
# DATA PREPROCESSING FOR QML
# ============================================================================

# Select relevant numerical and categorical features for prediction
# We'll use a mix of features that could influence survival

# Numerical features
numerical_features = ['Age', 'Height', 'Weight', 'Wins', 'Losses', 
                      'Public Favor', 'Mental Resilience', 'Battle Experience']

# Categorical features to encode
categorical_features = ['Category', 'Special Skills', 'Weapon of Choice', 
                        'Patron Wealth', 'Equipment Quality', 'Health Status']

# Create a working copy
df_work = gladiator_df.copy()

# Encode categorical variables
label_encoders = {}
for col in categorical_features:
    le = LabelEncoder()
    df_work[col + '_encoded'] = le.fit_transform(df_work[col].astype(str))
    label_encoders[col] = le

# Combine features
feature_cols = numerical_features + [col + '_encoded' for col in categorical_features]
X = df_work[feature_cols].values
y = df_work['Survived'].astype(int).values  # Convert True/False to 1/0

# Normalize features (important for quantum circuits)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# For quantum circuits, we need to limit the number of features
# Quantum circuits with many qubits become computationally expensive
# We'll use the top 8 most important features (for 8 qubits)
# In practice, you could use feature selection techniques

# Select top 8 features based on correlation with survival
correlations = []
for i, col in enumerate(feature_cols):
    corr = np.abs(np.corrcoef(X[:, i], y)[0, 1])
    correlations.append((i, corr, col))

correlations.sort(key=lambda x: x[1], reverse=True)
selected_indices = [idx for idx, _, _ in correlations[:8]]
selected_features = [name for _, _, name in correlations[:8]]

print("Selected features for quantum circuit:")
for i, (idx, corr, name) in enumerate(correlations[:8]):
    print(f"{i+1}. {name}: correlation = {corr:.4f}")

# Extract selected features
X_selected = X_scaled[:, selected_indices]

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X_selected, y, test_size=0.2, random_state=42, stratify=y
)

print(f"\nTraining set: {X_train.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")
print(f"Features per sample: {X_train.shape[1]}")


In [ ]:
# ============================================================================
# QUANTUM CIRCUIT DEFINITION
# ============================================================================

# Number of qubits = number of features
n_qubits = X_train.shape[1]
n_layers = 2  # Number of variational layers (more layers = more expressivity)

# Create a quantum device
# 'default.qubit' is PennyLane's built-in simulator
# For real quantum hardware, you could use 'qiskit.ibmq' or other providers
dev = qml.device('default.qubit', wires=n_qubits)

print(f"Quantum device: {dev.name}")
print(f"Number of qubits: {n_qubits}")
print(f"Number of layers: {n_layers}")

@qml.qnode(dev)
def quantum_circuit(features, weights):
    """
    Variational Quantum Circuit for classification.
    
    Args:
        features: Input features (classical data) - shape (n_qubits,)
        weights: Trainable parameters - shape (n_layers, n_qubits, 3)
                 Each layer has n_qubits rotations with 3 parameters (RX, RY, RZ)
    
    Returns:
        Expectation value of Pauli-Z operator on the first qubit
        This gives us a value between -1 and 1, which we'll interpret as a prediction
    """
    # STEP 1: Feature Encoding (Angle Encoding)
    # Encode classical features into quantum states by rotating qubits
    # Each feature value becomes a rotation angle
    for i in range(n_qubits):
        # Apply a rotation around Y-axis based on feature value
        # Using arctan to map features to [-π/2, π/2] range
        qml.RY(np.arctan(features[i]), wires=i)
    
    # STEP 2: Variational Layers (Parameterized Quantum Gates)
    # These layers contain trainable parameters that will be optimized
    for layer in range(n_layers):
        # Entangling layer: Create quantum correlations between qubits
        # This allows the circuit to learn complex relationships
        for i in range(n_qubits - 1):
            qml.CNOT(wires=[i, i + 1])  # Controlled-NOT gate creates entanglement
        
        # Rotation layer: Apply parameterized rotations
        # These are the "weights" that will be trained
        for i in range(n_qubits):
            qml.Rot(weights[layer, i, 0],  # Rotation around X-axis
                   weights[layer, i, 1],  # Rotation around Y-axis
                   weights[layer, i, 2],  # Rotation around Z-axis
                   wires=i)
    
    # STEP 3: Measurement
    # Measure the expectation value of Pauli-Z operator on first qubit
    # This gives us a real-valued output that we can use for classification
    return qml.expval(qml.PauliZ(0))

# Initialize random weights
# Shape: (n_layers, n_qubits, 3) - 3 rotation angles per qubit per layer
weights = pnp.random.uniform(0, 2 * np.pi, size=(n_layers, n_qubits, 3), requires_grad=True)

print(f"\nWeight shape: {weights.shape}")
print(f"Total trainable parameters: {np.prod(weights.shape)}")

# Test the circuit with a sample input
sample_features = X_train[0]
sample_output = quantum_circuit(sample_features, weights)
print(f"\nSample output (expectation value): {sample_output:.4f}")


In [ ]:
# ============================================================================
# QUANTUM CLASSIFIER MODEL
# ============================================================================

def quantum_classifier(weights, bias, features):
    """
    Quantum classifier that combines quantum circuit output with classical bias.
    
    Args:
        weights: Quantum circuit parameters
        bias: Classical bias term
        features: Input features
    
    Returns:
        Prediction value (before sigmoid activation)
    """
    # Get quantum circuit output
    quantum_output = quantum_circuit(features, weights)
    
    # Combine with bias and return
    # The quantum output is in [-1, 1], we scale it and add bias
    return quantum_output + bias

def cost_function(weights, bias, X, y):
    """
    Cost function for training the quantum classifier.
    Uses mean squared error between predictions and true labels.
    
    Args:
        weights: Quantum circuit parameters
        bias: Classical bias term
        X: Training features
        y: Training labels (0 or 1)
    
    Returns:
        Mean squared error
    """
    predictions = [quantum_classifier(weights, bias, x) for x in X]
    predictions = pnp.array(predictions)
    
    # Convert quantum output [-1, 1] to probability [0, 1]
    # Using sigmoid-like transformation: (x + 1) / 2
    probabilities = (predictions + 1) / 2
    
    # Mean squared error
    loss = pnp.mean((probabilities - y) ** 2)
    return loss

# Initialize bias
bias = pnp.array(0.0, requires_grad=True)

print("Quantum classifier model defined!")
print(f"Initial cost: {cost_function(weights, bias, X_train[:10], y_train[:10]):.4f}")


In [ ]:
# ============================================================================
# TRAINING THE QUANTUM CLASSIFIER
# ============================================================================

# Use PennyLane's optimizer (Adam optimizer)
opt = qml.AdamOptimizer(stepsize=0.1)

# Training parameters
n_epochs = 30
batch_size = 20  # Process data in batches to reduce computation time

# Store training history
cost_history = []

print("=" * 80)
print("TRAINING QUANTUM CLASSIFIER")
print("=" * 80)
print(f"Epochs: {n_epochs}")
print(f"Batch size: {batch_size}")
print(f"Training samples: {len(X_train)}")
print("\nTraining...")

# Training loop
for epoch in range(n_epochs):
    # Shuffle training data
    indices = np.random.permutation(len(X_train))
    X_shuffled = X_train[indices]
    y_shuffled = y_train[indices]
    
    epoch_cost = 0
    n_batches = len(X_train) // batch_size
    
    # Process in batches
    for batch_idx in range(n_batches):
        start_idx = batch_idx * batch_size
        end_idx = start_idx + batch_size
        
        X_batch = X_shuffled[start_idx:end_idx]
        y_batch = y_shuffled[start_idx:end_idx]
        
        # Define cost function for this batch
        def batch_cost(w, b):
            return cost_function(w, b, X_batch, y_batch)
        
        # Optimize weights and bias
        weights, bias = opt.step(batch_cost, weights, bias)
        
        # Calculate cost
        batch_cost_val = batch_cost(weights, bias)
        epoch_cost += batch_cost_val
    
    avg_cost = epoch_cost / n_batches
    cost_history.append(avg_cost)
    
    if (epoch + 1) % 5 == 0:
        print(f"Epoch {epoch + 1}/{n_epochs}, Cost: {avg_cost:.4f}")

print("\nTraining completed!")

# Plot training history
plt.figure(figsize=(10, 6))
plt.plot(cost_history, 'b-', linewidth=2)
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Cost (MSE)', fontsize=12)
plt.title('Quantum Classifier Training History', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.show()


In [ ]:
# ============================================================================
# EVALUATING THE QUANTUM CLASSIFIER
# ============================================================================

def predict(weights, bias, X):
    """
    Make predictions using the trained quantum classifier.
    
    Args:
        weights: Trained quantum circuit parameters
        bias: Trained bias term
        X: Features to predict on
    
    Returns:
        Binary predictions (0 or 1)
    """
    predictions = []
    for x in X:
        output = quantum_classifier(weights, bias, x)
        # Convert quantum output to probability
        prob = (output + 1) / 2
        # Threshold at 0.5
        predictions.append(1 if prob > 0.5 else 0)
    return np.array(predictions)

# Make predictions on test set
y_pred = predict(weights, bias, X_test)

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)

print("=" * 80)
print("QUANTUM CLASSIFIER RESULTS")
print("=" * 80)
print(f"\nTest Accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)")
print(f"\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=['Died', 'Survived']))

# Confusion matrix
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Died', 'Survived'],
            yticklabels=['Died', 'Survived'])
plt.title('Quantum Classifier Confusion Matrix', fontsize=14, fontweight='bold')
plt.ylabel('True Label', fontsize=12)
plt.xlabel('Predicted Label', fontsize=12)
plt.show()

# Compare with baseline (always predict majority class)
baseline_accuracy = max(y_test.mean(), 1 - y_test.mean())
print(f"\nBaseline Accuracy (majority class): {baseline_accuracy:.4f}")
print(f"Quantum Improvement: {accuracy - baseline_accuracy:.4f}")


# Part 3: Quantum Optimization - Finding Optimal Historical Topic Groupings

## Overview
We'll use the **Quantum Approximate Optimization Algorithm (QAOA)** to find optimal groupings of historical Wikipedia topics. This demonstrates how quantum computers can solve combinatorial optimization problems.

### Key Concepts:
- **QAOA**: A quantum algorithm for solving optimization problems
- **Max-Cut Problem**: Finding optimal partitions in a graph (we'll create a graph from topic similarities)
- **Cost Hamiltonian**: Quantum representation of the optimization objective
- **Variational Optimization**: Finding optimal parameters through classical optimization


In [ ]:
# ============================================================================
# PREPARING DATA FOR QUANTUM OPTIMIZATION
# ============================================================================

# For QAOA, we'll work with a smaller subset of Wikipedia entries
# QAOA works best with smaller graphs (due to current quantum hardware limitations)
n_topics = 20  # Number of topics to cluster

# Select top N topics by popularity/ranking
wiki_subset = wiki_df.nlargest(n_topics, 'popularity').copy()

print("=" * 80)
print("QUANTUM OPTIMIZATION: Historical Topic Clustering")
print("=" * 80)
print(f"Selected {n_topics} topics for optimization")
print(f"\nSelected topics:")
for i, title in enumerate(wiki_subset['title'].head(10), 1):
    print(f"{i}. {title}")

# Create a similarity graph based on text content
# We'll use simple keyword matching to create edges
# In practice, you could use more sophisticated NLP techniques

# Extract keywords from titles (simple approach)
def extract_keywords(text):
    """Extract keywords from text (simplified)."""
    if pd.isna(text):
        return []
    # Convert to lowercase and split
    words = str(text).lower().split()
    # Filter out very short words
    keywords = [w for w in words if len(w) > 3]
    return set(keywords)

# Build similarity matrix
similarity_matrix = np.zeros((n_topics, n_topics))

for i in range(n_topics):
    keywords_i = extract_keywords(wiki_subset.iloc[i]['title'])
    for j in range(i + 1, n_topics):
        keywords_j = extract_keywords(wiki_subset.iloc[j]['title'])
        # Jaccard similarity
        intersection = len(keywords_i & keywords_j)
        union = len(keywords_i | keywords_j)
        similarity = intersection / union if union > 0 else 0
        similarity_matrix[i, j] = similarity
        similarity_matrix[j, i] = similarity

# Create adjacency matrix for graph
# Connect topics with similarity > threshold
threshold = 0.1
adjacency_matrix = (similarity_matrix > threshold).astype(int)
np.fill_diagonal(adjacency_matrix, 0)  # No self-loops

# Count edges
n_edges = np.sum(adjacency_matrix) // 2
print(f"\nGraph statistics:")
print(f"  Nodes (topics): {n_topics}")
print(f"  Edges (connections): {n_edges}")
print(f"  Average degree: {np.sum(adjacency_matrix) / n_topics:.2f}")


In [ ]:
# ============================================================================
# QAOA IMPLEMENTATION FOR MAX-CUT PROBLEM
# ============================================================================

# Max-Cut Problem: Partition graph into two groups to maximize edges between groups
# This is a classic optimization problem that QAOA can solve efficiently

# Create quantum device
n_qubits_opt = n_topics
dev_opt = qml.device('default.qubit', wires=n_qubits_opt)

def maxcut_cost_hamiltonian(adjacency_matrix):
    """
    Create the cost Hamiltonian for Max-Cut problem.
    
    Max-Cut: Maximize the number of edges between two partitions.
    Cost Hamiltonian: Sum over edges (1 - Z_i Z_j) / 2
    where Z_i, Z_j are Pauli-Z operators on qubits i and j
    
    Args:
        adjacency_matrix: Graph adjacency matrix
    
    Returns:
        Cost Hamiltonian as a PennyLane observable and list of edges
    """
    # List of observables (one for each edge)
    observables = []
    coeffs = []
    edges = []  # Store edges for easier circuit construction
    
    for i in range(n_qubits_opt):
        for j in range(i + 1, n_qubits_opt):
            if adjacency_matrix[i, j] == 1:  # If there's an edge
                # Cost for edge (i,j): (1 - Z_i Z_j) / 2
                # We want to maximize this, so minimize its negative
                obs = qml.PauliZ(i) @ qml.PauliZ(j)
                observables.append(obs)
                coeffs.append(-0.5)  # Negative because we'll minimize
                edges.append((i, j))
    
    # Constant term (doesn't affect optimization)
    constant = np.sum(adjacency_matrix) / 2
    
    return qml.Hamiltonian(coeffs, observables), constant, edges

# Create cost Hamiltonian
cost_hamiltonian, constant, edges = maxcut_cost_hamiltonian(adjacency_matrix)

print(f"Cost Hamiltonian created with {len(edges)} terms (edges)")
print(f"Constant term: {constant}")

def mixer_hamiltonian(n_qubits):
    """
    Create the mixer Hamiltonian for QAOA.
    Mixer: Sum of Pauli-X operators on all qubits
    This allows the algorithm to explore the solution space.
    
    Args:
        n_qubits: Number of qubits
    
    Returns:
        Mixer Hamiltonian
    """
    coeffs = []
    observables = []
    for i in range(n_qubits):
        coeffs.append(1.0)
        observables.append(qml.PauliX(i))
    return qml.Hamiltonian(coeffs, observables)

mixer_hamiltonian = mixer_hamiltonian(n_qubits_opt)
print(f"Mixer Hamiltonian created")


In [ ]:
# ============================================================================
# QAOA CIRCUIT
# ============================================================================

def qaoa_layer(gamma, beta, edges_list):
    """
    One layer of QAOA circuit.
    
    QAOA alternates between:
    1. Applying cost Hamiltonian (with parameter gamma)
    2. Applying mixer Hamiltonian (with parameter beta)
    
    Args:
        gamma: Parameter for cost Hamiltonian evolution
        beta: Parameter for mixer Hamiltonian evolution
        edges_list: List of edges (i, j) in the graph
    """
    # Apply cost Hamiltonian evolution
    # Our cost Hamiltonian is: H = sum over edges (-0.5 * Z_i * Z_j)
    # We want to apply: exp(-i*gamma*H) = exp(i*gamma*0.5*sum(Z_i*Z_j))
    # For each edge, apply exp(i*gamma*0.5*Z_i*Z_j)
    # Using CNOT-RZ-CNOT decomposition: exp(i*theta*Z_i*Z_j) = CNOT RZ(-2*theta) CNOT
    for i, j in edges_list:
        # CNOT-RZ-CNOT decomposition for exp(i*gamma*0.5*Z_i*Z_j)
        # theta = gamma*0.5, so rotation angle = -2*theta = -gamma
        qml.CNOT(wires=[i, j])
        qml.RZ(-gamma, wires=j)
        qml.CNOT(wires=[i, j])
    
    # Apply mixer Hamiltonian evolution
    # Mixer is sum of Pauli-X: exp(-i*beta*sum(X_i)) = product of RX(2*beta)
    for i in range(n_qubits_opt):
        qml.RX(2 * beta, wires=i)

def qaoa_circuit(params, cost_ham, mixer_ham):
    """
    Full QAOA circuit.
    
    Args:
        params: Array of parameters [gamma_1, beta_1, gamma_2, beta_2, ...]
        cost_ham: Cost Hamiltonian (for measurement)
        mixer_ham: Mixer Hamiltonian (not used directly, kept for API consistency)
    
    Returns:
        Expectation value of cost Hamiltonian
    """
    # Initialize in uniform superposition |+⟩^⊗n
    for i in range(n_qubits_opt):
        qml.Hadamard(wires=i)
    
    # Apply QAOA layers
    p = len(params) // 2  # Number of layers
    for i in range(p):
        gamma = params[2 * i]
        beta = params[2 * i + 1]
        qaoa_layer(gamma, beta, edges)
    
    # Measure expectation value of cost Hamiltonian
    return qml.expval(cost_ham)

# Create QAOA QNode
qaoa_qnode = qml.QNode(qaoa_circuit, dev_opt)

# Number of QAOA layers (depth)
p = 2  # More layers = better approximation, but more parameters to optimize

# Initialize parameters randomly
init_params = pnp.random.uniform(0, 2 * np.pi, size=2 * p, requires_grad=True)

print(f"QAOA circuit created")
print(f"  Number of layers (p): {p}")
print(f"  Number of parameters: {len(init_params)}")
print(f"  Initial parameters: {init_params}")


In [ ]:
# ============================================================================
# OPTIMIZING QAOA PARAMETERS
# ============================================================================

def qaoa_cost(params):
    """
    Cost function for QAOA optimization.
    We want to minimize the expectation value of the cost Hamiltonian.
    
    Args:
        params: QAOA parameters
    
    Returns:
        Expectation value (to be minimized)
    """
    return qaoa_qnode(params, cost_hamiltonian, mixer_hamiltonian)

# Optimize QAOA parameters
opt_qaoa = qml.AdamOptimizer(stepsize=0.1)

print("=" * 80)
print("OPTIMIZING QAOA PARAMETERS")
print("=" * 80)
print("This may take a few minutes...")

n_iterations = 50
params = init_params
cost_history_qaoa = []

for i in range(n_iterations):
    # Optimize
    params, cost_val = opt_qaoa.step_and_cost(qaoa_cost, params)
    cost_history_qaoa.append(cost_val)
    
    if (i + 1) % 10 == 0:
        print(f"Iteration {i + 1}/{n_iterations}, Cost: {cost_val:.4f}")

print("\nOptimization completed!")
print(f"Final cost: {cost_history_qaoa[-1]:.4f}")

# Plot optimization history
plt.figure(figsize=(10, 6))
plt.plot(cost_history_qaoa, 'r-', linewidth=2)
plt.xlabel('Iteration', fontsize=12)
plt.ylabel('Cost (Expectation Value)', fontsize=12)
plt.title('QAOA Optimization History', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.show()


In [ ]:
# ============================================================================
# EXTRACTING OPTIMAL SOLUTION
# ============================================================================

# Sample from the optimized QAOA circuit to get the solution
n_samples = 1000

def sample_solution(params, n_samples):
    """
    Sample bitstrings from QAOA circuit and find the best solution.
    
    Args:
        params: Optimized QAOA parameters
        n_samples: Number of samples to draw
    
    Returns:
        Best bitstring and its cost
    """
    # Create a circuit that measures in computational basis
    @qml.qnode(dev_opt)
    def measurement_circuit(params):
        # Apply QAOA circuit
        for i in range(n_qubits_opt):
            qml.Hadamard(wires=i)
        
        p = len(params) // 2
        for i in range(p):
            gamma = params[2 * i]
            beta = params[2 * i + 1]
            qaoa_layer(gamma, beta, edges)
        
        # Measure all qubits
        return [qml.sample(qml.PauliZ(i)) for i in range(n_qubits_opt)]
    
    # Sample bitstrings
    samples = []
    for _ in range(n_samples):
        sample = measurement_circuit(params)
        # Convert from {-1, +1} to {0, 1}
        bitstring = [(1 - s) // 2 for s in sample]
        samples.append(bitstring)
    
    # Calculate cost for each sample
    def calculate_cut_cost(bitstring):
        """Calculate the cut value for a given partition."""
        cut_value = 0
        for i in range(n_qubits_opt):
            for j in range(i + 1, n_qubits_opt):
                if adjacency_matrix[i, j] == 1:
                    # Edge is cut if nodes are in different partitions
                    if bitstring[i] != bitstring[j]:
                        cut_value += 1
        return cut_value
    
    # Find best solution
    best_bitstring = None
    best_cost = -1
    
    for bitstring in samples:
        cost = calculate_cut_cost(bitstring)
        if cost > best_cost:
            best_cost = cost
            best_bitstring = bitstring
    
    return best_bitstring, best_cost

print("Sampling solutions from optimized QAOA circuit...")
best_solution, best_cut_value = sample_solution(params, n_samples)

print(f"\nOptimal partition found!")
print(f"Cut value (edges between partitions): {best_cut_value} out of {n_edges} total edges")
print(f"Cut ratio: {best_cut_value / n_edges:.2%}")

# Display the partition
group_0 = [i for i, bit in enumerate(best_solution) if bit == 0]
group_1 = [i for i, bit in enumerate(best_solution) if bit == 1]

print(f"\nGroup 0 ({len(group_0)} topics):")
for idx in group_0:
    print(f"  - {wiki_subset.iloc[idx]['title']}")

print(f"\nGroup 1 ({len(group_1)} topics):")
for idx in group_1:
    print(f"  - {wiki_subset.iloc[idx]['title']}")


In [ ]:
# ============================================================================
# VISUALIZATION OF OPTIMIZATION RESULTS
# ============================================================================

# Visualize the graph partition
import matplotlib.patches as mpatches

# Create a simple visualization
fig, ax = plt.subplots(figsize=(12, 8))

# Plot nodes
node_positions = {}
n_per_row = int(np.ceil(np.sqrt(n_topics)))
for i in range(n_topics):
    row = i // n_per_row
    col = i % n_per_row
    node_positions[i] = (col, -row)

# Color nodes by partition
colors = ['lightblue' if best_solution[i] == 0 else 'lightcoral' for i in range(n_topics)]

# Draw edges
for i in range(n_topics):
    for j in range(i + 1, n_topics):
        if adjacency_matrix[i, j] == 1:
            x1, y1 = node_positions[i]
            x2, y2 = node_positions[j]
            # Color edge based on whether it's cut
            edge_color = 'red' if best_solution[i] != best_solution[j] else 'gray'
            ax.plot([x1, x2], [y1, y2], color=edge_color, linewidth=1, alpha=0.5)

# Draw nodes
for i in range(n_topics):
    x, y = node_positions[i]
    ax.scatter(x, y, s=500, c=colors[i], edgecolors='black', linewidth=2, zorder=3)
    # Add label
    title = wiki_subset.iloc[i]['title'][:20]  # Truncate long titles
    ax.text(x, y, f'{i}', ha='center', va='center', fontsize=8, fontweight='bold')

ax.set_title('QAOA Optimal Partition of Historical Topics', fontsize=14, fontweight='bold')
ax.axis('off')

# Add legend
group0_patch = mpatches.Patch(color='lightblue', label='Group 0')
group1_patch = mpatches.Patch(color='lightcoral', label='Group 1')
ax.legend(handles=[group0_patch, group1_patch], loc='upper right')

plt.tight_layout()
plt.show()

print("\n" + "=" * 80)
print("QUANTUM OPTIMIZATION COMPLETE!")
print("=" * 80)
print("\nThe QAOA algorithm has found an optimal partition of historical topics")
print("that maximizes the number of edges (similarities) between the two groups.")
print("\nThis demonstrates how quantum algorithms can solve combinatorial")
print("optimization problems that are difficult for classical computers!")


# Summary and Conclusions

## What We've Demonstrated:

### 1. Quantum Machine Learning (QML)
- **Variational Quantum Classifier (VQC)** for predicting gladiator survival
- Used quantum feature encoding and variational layers
- Achieved classification accuracy on historical gladiator data
- Demonstrated how quantum circuits can learn patterns in classical data

### 2. Quantum Optimization
- **Quantum Approximate Optimization Algorithm (QAOA)** for finding optimal partitions
- Solved Max-Cut problem on historical Wikipedia topic similarity graph
- Found optimal groupings that maximize inter-group connections
- Demonstrated quantum advantage potential for combinatorial optimization

## Key Quantum Computing Concepts Used:

1. **Quantum Superposition**: Qubits can exist in multiple states simultaneously
2. **Quantum Entanglement**: Qubits can be correlated in ways impossible classically
3. **Variational Quantum Algorithms**: Hybrid quantum-classical optimization
4. **Quantum Measurement**: Extracting classical information from quantum states

## Future Directions:

- Scale to larger datasets with more qubits
- Use real quantum hardware (IBM Quantum, IonQ, etc.)
- Explore other QML architectures (Quantum Neural Networks, Quantum Kernels)
- Apply to other optimization problems (Traveling Salesman, Portfolio Optimization)
